In [2]:
import pandas as pd
import numpy as np

# Load the data
data = pd.read_csv('/home/bettimj/gamazon_rotation/mod_core-bed/hi-c/t2d_gwas/t2d_compiled_core-bed.txt', sep = "\t")

In [3]:
data

,SNP,disease_tissues,gene_overlap,tissue_gene_contact,gene_contact,gene_contact_total
0,rs10000502,"adipose, digestive, endocrine, liver, muscle, ...",MAML3,"esc_GUCY1B1,ipsc_MGST2,esc_deriv_MGST2,esc_MGS...","GUCY1B1, MGST2, ELF2, NDUFC1, TBC1D9, INPP4B,...","GUCY1B1, MGST2, MGST2, MGST2, ELF2, ELF2, NDU..."
1,rs10000702,"adipose, digestive, endocrine, liver, muscle, ...",PDGFC,"neurosph_CD38,brain_CD38,kidney_MCUB,neurosph_...","CD38, MCUB, TLL1, ARAP2, TNIP3, FNIP2, GATB, ...","CD38, CD38, MCUB, TLL1, TLL1, ARAP2, ARAP2, T..."
2,rs1000237,"adipose, digestive, endocrine, liver, muscle, ...",GATAD2A,"stromal_UPF1,bone_UPF1,heart_UPF1,epithelial_U...","UPF1, REX1BD, CRLF1, MARK4, MATK, MPND, NFIX,...","UPF1, UPF1, UPF1, UPF1, REX1BD, REX1BD, CRLF1..."
3,rs1000285,"adipose, digestive, endocrine, liver, muscle, ...",NDUFAF6,NaN,NaN,NaN
4,rs1000286,"adipose, digestive, endocrine, liver, muscle, ...",NDUFAF6,NaN,NaN,NaN
...,...,...,...,...,...,...
18686,rs999475,"adipose, digestive, endocrine, liver, muscle, ...",UBE2Z,"stromal_LASP1,liver_SNX11,bone_SNX11,stromal_S...","LASP1, SNX11, SARM1, CDC27, COPZ2, ITGA3, OSB...","LASP1, SNX11, SNX11, SNX11, SARM1, CDC27, COP..."
18687,rs9997745,"adipose, digestive, endocrine, liver, muscle, ...","ACSL1,AC084871.1","hsc_and_b_cell_ADD1,hsc_and_b_cell_ACSL1,lymph...","ADD1, ACSL1, SORBS2, CEP44, ENPP6, TENM3","ADD1, ACSL1, ACSL1, SORBS2, CEP44, ENPP6, TENM3"
18688,rs9997958,"adipose, digestive, endocrine, liver, muscle, ...",PDGFC,"epithelial_FNIP2,epithelial_ARHGAP10,epithelia...","FNIP2, ARHGAP10, TMEM33, KIAA1211, GAB1, KLHL...","FNIP2, ARHGAP10, TMEM33, KIAA1211, GAB1, KLHL..."
18689,rs9998551,"adipose, digestive, endocrine, liver, muscle, ...",NaN,"epithelial_RUFY3,epithelial_UBA6,epithelial_PO...","RUFY3, UBA6, POLR2B, SLC4A4, GNRHR, ODAM, SMR...","RUFY3, UBA6, POLR2B, SLC4A4, SLC4A4, GNRHR, O..."


In [4]:
import pandas as pd
import numpy as np

# Function to generate the disease tissue-relevant credible set and compute the scores
def compute_core_bed_scores(row):
    # Step 1: Construct the credible set
    if pd.notnull(row["disease_tissues"]) and pd.notnull(row["tissue_gene_contact"]):
        disease_tissues = {tissue.strip() for tissue in row["disease_tissues"].split(',')}
        gene_tissue_pairs = row["tissue_gene_contact"].split(',')
        
        relevant_genes = []
        for pair in gene_tissue_pairs:
            tissue_gene_split = pair.rsplit('_', 1)
            if len(tissue_gene_split) == 2:
                tissue, gene = tissue_gene_split
                if tissue in disease_tissues:
                    relevant_genes.append(gene.strip())
        
        credible_set = set(relevant_genes) if relevant_genes else None
    else:
        credible_set = None

    # Step 2: If the credible set is empty, check the gene_overlap column
    if credible_set is None and pd.notnull(row["gene_overlap"]):
        overlap_genes = {gene.strip() for gene in row["gene_overlap"].split(',')}
        credible_set = overlap_genes if overlap_genes else None

    # If the credible set is still None, return None
    if credible_set is None:
        return None

    # Step 3: Calculate contact instances C(g)
    gene_counts = {}
    if isinstance(row["gene_contact_total"], str):
        for gene in credible_set:
            gene_counts[gene] = row["gene_contact_total"].count(gene)
    else:
        gene_counts = {gene: 0 for gene in credible_set}  # If data is missing or not a string, count as 0

    # Step 4: Rank genes and apply exponential decay
    ranked_genes = sorted(gene_counts, key=gene_counts.get, reverse=True)
    gene_ranks = {gene: rank for rank, gene in enumerate(ranked_genes, 1)}
    rank_weights = {gene: np.exp(-0.3 * (rank - 1)) for gene, rank in gene_ranks.items()}

    # Step 5: Calculate base score B(g)
    base_scores = {
        gene: 1 / sum(1 for g in credible_set if gene_ranks[g] == gene_ranks[gene]) 
        for gene in credible_set
    }

    # Step 6: Compute CoRE-BED score S(g)
    initial_scores = {
        gene: base_scores[gene] * rank_weights[gene] 
        for gene in credible_set
    }

    # Step 7: Adjust for overlap w(g)
    overlap_genes = set(row["gene_overlap"].split(',')) if pd.notnull(row["gene_overlap"]) else set()
    final_scores = {
        gene: score * (2 if gene in overlap_genes else 1) 
        for gene, score in initial_scores.items()
    }

    # Step 8: Normalize the final score F_normalized(g) using softmax
    exp_scores = {gene: np.exp(score) for gene, score in final_scores.items()}
    sum_exp_scores = sum(exp_scores.values()) if exp_scores else 1
    softmax_scores = {
        gene: exp_scores[gene] / sum_exp_scores 
        for gene in exp_scores
    }

    return softmax_scores

# Apply the function to compute scores and store the results
data['gene_scores'] = data.apply(compute_core_bed_scores, axis=1)

# Initialize an empty list to store highest-scoring genes
highest_scoring_genes_list = []

# Extract the highest-scoring gene for each SNP
for index, row in data.iterrows():
    gene_scores = row['gene_scores']
    if gene_scores:
        # Get the highest-scoring gene and its score
        highest_scoring_gene = max(gene_scores, key=gene_scores.get)
        highest_score = gene_scores[highest_scoring_gene]
        # Append to the list as a dictionary
        highest_scoring_genes_list.append({
            'SNP': row['SNP'],
            'Highest_Scoring_Gene': highest_scoring_gene,
            'CoRE_BED_Score': highest_score
        })

# Convert the list to a DataFrame
highest_scoring_genes = pd.DataFrame(highest_scoring_genes_list)

# Optionally, print out the highest scores for each SNP
for index, row in highest_scoring_genes.iterrows():
    print(f"SNP: {row['SNP']}, Highest Scoring Gene: {row['Highest_Scoring_Gene']}, CoRE-BED Score: {row['CoRE_BED_Score']}")

# Save the highest-scoring genes to a CSV file
highest_scoring_genes.to_csv('/home/bettimj/gamazon_rotation/mod_core-bed/hi-c/t2d_gwas/t2d_top_scoring_genes.csv', index=False)

SNP: rs10000502, Highest Scoring Gene: MAML3, CoRE-BED Score: 1.0
SNP: rs10000702, Highest Scoring Gene: PDGFC, CoRE-BED Score: 0.12115117373016455
SNP: rs1000237, Highest Scoring Gene: GATAD2A, CoRE-BED Score: 0.04589750063334548
SNP: rs1000285, Highest Scoring Gene: NDUFAF6, CoRE-BED Score: 1.0
SNP: rs1000286, Highest Scoring Gene: NDUFAF6, CoRE-BED Score: 1.0
SNP: rs10004357, Highest Scoring Gene: PPP3CA, CoRE-BED Score: 1.0
SNP: rs1000454, Highest Scoring Gene: CMIP, CoRE-BED Score: 1.0
SNP: rs10005752, Highest Scoring Gene: GAK, CoRE-BED Score: 1.0
SNP: rs10007719, Highest Scoring Gene: PDGFC, CoRE-BED Score: 1.0
SNP: rs10010325, Highest Scoring Gene: TET2, CoRE-BED Score: 1.0
SNP: rs10011838, Highest Scoring Gene: FBXW7, CoRE-BED Score: 0.5644351444092433
SNP: rs10012045, Highest Scoring Gene: FAM13A, CoRE-BED Score: 0.06209792806022947
SNP: rs10012624, Highest Scoring Gene: FAM13A, CoRE-BED Score: 0.06209792806022947
SNP: rs10014011, Highest Scoring Gene: PDGFC, CoRE-BED Score: 

In [7]:
import pandas as pd
import numpy as np

# Function to generate the disease tissue-relevant credible set and compute the scores
def compute_core_bed_scores(row):
    # Step 1: Construct the credible set
    if pd.notnull(row["disease_tissues"]) and pd.notnull(row["tissue_gene_contact"]):
        disease_tissues = {tissue.strip() for tissue in row["disease_tissues"].split(',')}
        gene_tissue_pairs = row["tissue_gene_contact"].split(',')
        
        relevant_genes = []
        for pair in gene_tissue_pairs:
            tissue_gene_split = pair.rsplit('_', 1)
            if len(tissue_gene_split) == 2:
                tissue, gene = tissue_gene_split
                if tissue in disease_tissues:
                    relevant_genes.append(gene.strip())
        
        credible_set = set(relevant_genes) if relevant_genes else None
    else:
        credible_set = None

    # Step 2: If the credible set is empty, check the gene_overlap column
    if credible_set is None and pd.notnull(row["gene_overlap"]):
        overlap_genes = {gene.strip() for gene in row["gene_overlap"].split(',')}
        credible_set = overlap_genes if overlap_genes else None

    # If the credible set is still None, return None
    if credible_set is None:
        return None

    # Step 3: Calculate contact instances C(g)
    gene_counts = {}
    if isinstance(row["gene_contact_total"], str):
        for gene in credible_set:
            gene_counts[gene] = row["gene_contact_total"].count(gene)
    else:
        gene_counts = {gene: 0 for gene in credible_set}  # If data is missing or not a string, count as 0

    # Step 4: Rank genes and apply exponential decay
    ranked_genes = sorted(gene_counts, key=gene_counts.get, reverse=True)
    gene_ranks = {gene: rank for rank, gene in enumerate(ranked_genes, 1)}
    rank_weights = {gene: np.exp(-0.3 * (rank - 1)) for gene, rank in gene_ranks.items()}

    # Step 5: Calculate base score B(g)
    base_scores = {
        gene: 1 / sum(1 for g in credible_set if gene_ranks[g] == gene_ranks[gene]) 
        for gene in credible_set
    }

    # Step 6: Compute CoRE-BED score S(g)
    initial_scores = {
        gene: base_scores[gene] * rank_weights[gene] 
        for gene in credible_set
    }

    # Step 7: Adjust for overlap w(g)
    overlap_genes = set(row["gene_overlap"].split(',')) if pd.notnull(row["gene_overlap"]) else set()
    final_scores = {
        gene: score * (2 if gene in overlap_genes else 1) 
        for gene, score in initial_scores.items()
    }

    # Step 8: Normalize the final score using softmax
    exp_scores = {gene: np.exp(score) for gene, score in final_scores.items()}
    sum_exp_scores = sum(exp_scores.values()) if exp_scores else 1
    softmax_scores = {
        gene: exp_scores[gene] / sum_exp_scores 
        for gene in exp_scores
    }

    return softmax_scores

# Apply the function to compute scores and store the results
data['gene_scores'] = data.apply(compute_core_bed_scores, axis=1)

# Initialize an empty list to store highest-scoring genes
highest_scoring_genes_list = []
all_gene_scores_list = []

# Extract the highest-scoring gene for each SNP and save all scores
for index, row in data.iterrows():
    gene_scores = row['gene_scores']
    if gene_scores:
        # Get the highest-scoring gene and its score
        highest_scoring_gene = max(gene_scores, key=gene_scores.get)
        highest_score = gene_scores[highest_scoring_gene]
        # Append to the list as a dictionary
        highest_scoring_genes_list.append({
            'SNP': row['SNP'],
            'Highest_Scoring_Gene': highest_scoring_gene,
            'CoRE_BED_Score': highest_score
        })
        # Save all gene scores for this SNP
        for gene, score in gene_scores.items():
            all_gene_scores_list.append({
                'SNP': row['SNP'],
                'Gene': gene,
                'CoRE_BED_Score': score
            })

# Convert the lists to DataFrames
highest_scoring_genes = pd.DataFrame(highest_scoring_genes_list)
all_gene_scores = pd.DataFrame(all_gene_scores_list)

# Optionally, print out the highest scores for each SNP
for index, row in highest_scoring_genes.iterrows():
    print(f"SNP: {row['SNP']}, Highest Scoring Gene: {row['Highest_Scoring_Gene']}, CoRE-BED Score: {row['CoRE_BED_Score']}")

# Save the highest-scoring genes to a CSV file
highest_scoring_genes.to_csv('/home/bettimj/gamazon_rotation/mod_core-bed/hi-c/t2d_gwas/t2d_top_scoring_genes.csv', index=False)

# Make sure CoRE_BED_Score is numeric
all_gene_scores['CoRE_BED_Score'] = pd.to_numeric(all_gene_scores['CoRE_BED_Score'], errors='coerce')

# Make sure the SNP column keeps the order from the input dataset
all_gene_scores['SNP'] = pd.Categorical(all_gene_scores['SNP'], categories=data['SNP'], ordered=True)

# Sort the dataframe by SNP and CoRE_BED_Score (in descending order)
all_gene_scores = all_gene_scores.sort_values(by=['SNP', 'CoRE_BED_Score'], ascending=[True, False])

# Save the sorted DataFrame
all_gene_scores.to_csv('/home/bettimj/gamazon_rotation/mod_core-bed/hi-c/t2d_gwas/t2d_all_gene_scores.csv', index=False)

SNP: rs10000502, Highest Scoring Gene: MAML3, CoRE-BED Score: 1.0
SNP: rs10000702, Highest Scoring Gene: PDGFC, CoRE-BED Score: 0.12115117373016455
SNP: rs1000237, Highest Scoring Gene: GATAD2A, CoRE-BED Score: 0.04589750063334548
SNP: rs1000285, Highest Scoring Gene: NDUFAF6, CoRE-BED Score: 1.0
SNP: rs1000286, Highest Scoring Gene: NDUFAF6, CoRE-BED Score: 1.0
SNP: rs10004357, Highest Scoring Gene: PPP3CA, CoRE-BED Score: 1.0
SNP: rs1000454, Highest Scoring Gene: CMIP, CoRE-BED Score: 1.0
SNP: rs10005752, Highest Scoring Gene: GAK, CoRE-BED Score: 1.0
SNP: rs10007719, Highest Scoring Gene: PDGFC, CoRE-BED Score: 1.0
SNP: rs10010325, Highest Scoring Gene: TET2, CoRE-BED Score: 1.0
SNP: rs10011838, Highest Scoring Gene: FBXW7, CoRE-BED Score: 0.5644351444092433
SNP: rs10012045, Highest Scoring Gene: FAM13A, CoRE-BED Score: 0.06209792806022947
SNP: rs10012624, Highest Scoring Gene: FAM13A, CoRE-BED Score: 0.06209792806022947
SNP: rs10014011, Highest Scoring Gene: PDGFC, CoRE-BED Score: 